In [55]:
import pandas as pd
import numpy as np

## import base classifiers
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

## to measure run-times
import time
from datetime import timedelta

## the usual tools
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse

## to save and load models
import pickle

In [56]:
#Load all the datasets
year_list = [2008,2012,2016,2020]
df_list = [pd.read_csv(f'../Master/clean_data/data{year}.csv') for year in year_list]

In [57]:
#Here all the features we use:
all_features = ['labor%', 'armed%', 'empl%', 'unempl%', 'poverty%', 'income_median', 'income_percapita',
'income_10%', 'income_10-15%', 'income_15-25%', 'income_25%', 'single%',
'married%', 'marital_ratio', 'sepdiv%', 'widow%', 'edu_low%',
'edu_mid%', 'edu_high%', 'edulow_age18%', 'edulow_age45%',
'edulow_age65%', 'edumid_age18%', 'edumid_age45%', 'edumid_age65%',
'eduhigh_age18%', 'eduhigh_age45%', 'eduhigh_age65%', 'age_18%',
'age_45%', 'age_65%', 'wht%', 'wht_m%', 'wht_f%', 'blk%', 'blk_m%',
'blk_f%', 'other%', 'other_m%', 'other_f%', 'native%', 'hispanic%',
'pop_density', 'houses_density', 'pop_tot', 'area_sqmiles']

In [58]:
#Create lists of X and y (by year); we will partition these in various
#ways to carry out model-selection (via cross validation)
X_list = [df[all_features] for df in df_list]
y_list = [df['vote_diff%'] for df in df_list]

In [59]:
#Create 5 Kfold objects with different random states (all nice primes!)
kfold_list = [KFold(n_splits = 10,
                    shuffle = True,
                    random_state = r_state) for r_state in [13,17,43,61,101]]

#Create a list with four entries (one per election year)
#Each entry consists of the 5*10=50 CV splits coming from the 5 kfold objects
cv_list = []
for i in range(4):
    ith_cv = []
    X = X_list[i]
    y = y_list[i]
    for n,kfold in enumerate(kfold_list):
        #Generate list of 10 (X_train,X_test) pairs using the n-th kffold object
        for train_index,test_index in kfold.split(df_list[i]):
            X_tr = X.iloc[train_index]
            X_te = X.iloc[test_index]
            y_tr = y.iloc[train_index]
            y_te = y.iloc[test_index]
            ith_cv.append((X_tr,y_tr,X_te,y_te))
    cv_list.append(ith_cv)

In [60]:
#List of subsets of [0,1,2,3]
#For each subset, we will train our models on the union of 
#all the data from the corresponding years
subset3_list = [[0,1,2],[0,1,3],[0,2,3],[1,2,3]]
subset2_list = [[0,1],[0,2],[0,3],
             [2,3],[1,3],[1,2]]
subset1_list = [[3],[2],[1],[0]]

In [61]:
#The next method inputs a subset of [0,1,2,3], and it concatenates
#the CV splits correpsonding to those datasets.
#Thus, the output consists of 50 CV splits of the union of these datasets.
#This is done to ensure that each CV split draws equally from all 
#the years corresponding to that subset
def concat_cv(subset):
    concat_cv_list = []
    scaler = StandardScaler()
    for n in range(50):     #range over the 50 (train,test) 
        #Concatenate the nth train,test pair of the i-th year, for all i in subset
        X_tr = pd.concat([cv_list[i][n][0] for i in subset])
        y_tr = pd.concat([cv_list[i][n][1] for i in subset])
        X_te = pd.concat([cv_list[i][n][2] for i in subset])
        y_te = pd.concat([cv_list[i][n][3] for i in subset])
        #Scale the train,test pairs
        scaler.fit_transform(X_tr)
        scaler.transform(X_te)
        concat_cv_list.append((X_tr,y_tr,X_te,y_te))
    return concat_cv_list

#A simple and useful method to obtain the desired (X,y) depending
#on the subset of years we are training on.
def concat_main(subset):
    if subset in subset3_list + subset2_list:
        return(pd.concat([X_list[i] for i in subset]),
               pd.concat([y_list[i] for i in subset]))
    else:
        i = subset[0]
        return X_list[i],y_list[i]

In [62]:
#The 'vote_diff%' feature (for a particular county) is positive if 
#republicans win, and it is negative if Democrats win. Thus, the predicted
#vote difference fails to predict the winner exactly when it differs from 
#the true vote difference in sign; we use this fact to compute the accuracy
#of our predicted vote difference.
def vote_diff_acc(y_pred,y_true):
    y = y_pred * y_true
    return len(y[y >= 0])/len(y)

#The next method is used to tune hyperparameters.
#For each of the three models, it fits and computes the CV accuracy
#score as the corresponding hyperparameter ranges over a prescribed set of
#values. Then, it chooses the model with best mean CV accuracy and
#returns the associated hyperparameter value and some accuracy statistics
#(in the form of a dictionary)
def my_cv_scorer(subset,   
                 model,     #'ridge', 'knn', or 'rfr'
                 feature_list=all_features):    #optional: choose some subset of features

    if model=='ridge':
        param = 'alpha'
        param_list = [0.05*k for k in range(20)]
        model_list = [Ridge(alpha=alpha,
                            solver='lsqr') for alpha in param_list]
        
    elif model=='knn':
        param = 'k'
        param_list = range(3,16*(4 - len(subset)) - 1,2)
        model_list = [KNeighborsRegressor(k) for k in param_list]
        
    elif model=='rfr':
        param = 'max_features'
        if len(subset)==1:
            param_list = [0.6,0.7,0.8,0.9]
        elif len(subset)==2:
            param_list = [0.4,0.5,0.6,0.7]
        else: 
            param_list = [0.3,0.4,0.5,0.6]
        model_list = [RandomForestRegressor(n_estimators=100*len(subset),
                                            criterion='squared_error',
                                            max_depth=15,
                                            max_features=n,
                                            n_jobs=-1,
                                            bootstrap=True,
                                            random_state=42) for n in param_list]
    
    mse_list = []
    avgacc_list = []
    lowestacc_list = []
    highestacc_list = []

    #get list of cv splits
    concat_cv_list = [(X_tr[feature_list],y_tr,X_te[feature_list],y_te) for X_tr,y_tr,X_te,y_te in concat_cv(subset)]

    for modl in model_list:
        scores_cv = []
        mses_cv = []
        
        for X_train,y_train, X_test,y_test in concat_cv_list:
            modl.fit(X_train,y_train)                           #fit on the train set
            my_pred = modl.predict(X_test)                      #predict on the test set
            scores_cv.append(vote_diff_acc(y_test,my_pred))     #Compute accuracy score, add to list of scores     
            mses_cv.append(mse(y_test,my_pred))                 #Compute mse, add to list of mses for fixed k

        avgacc_list.append(round(np.mean(scores_cv),5))          #Add average CV accuracy
        mse_list.append(round(np.mean(mses_cv),5))              #Add average CV mse
        lowestacc_list.append(round(np.min(scores_cv),5))       #Add lowest CV accuracy
        highestacc_list.append(round(np.max(scores_cv),5))      #Add highest CV accuracy

    index = np.argmax(avgacc_list)
    best_model = model_list[index]

    if model =='ridge':
        fimp_df = pd.DataFrame({'feature':feature_list,
                                'importance': best_model.coef_,
                                'imp_abs': np.abs(best_model.coef_)}).sort_values(by='imp_abs',ascending=False,ignore_index=True)
        fimp_df.drop(inplace=True,columns=['imp_abs'])
    elif model=='rfr':
        fimp_df = pd.DataFrame({'feature': feature_list,
                                'importance': best_model.feature_importances_}).sort_values(by='importance',ascending=False,ignore_index=True)
    else:
        fimp_df = pd.DataFrame({'feature': feature_list,
                                'importance': 0})

    cv_results = {param: param_list[index],
                  'mse': mse_list[index],
                  'Average accuracy': avgacc_list[index],
                  'Highest accuracy': highestacc_list[index],
                  'Lowest accuracy': lowestacc_list[index],
                  'Features used': fimp_df['feature'],
                  'Importance': fimp_df['importance']}
    return cv_results

In [63]:
#The next method carries out feature selection for ridge linear regression
#The algorithm is recursive feature elimination with cross-validation
#(using the my_cv_scorer method)
def ridge_selector(subset):
    best_alpha_list = []
    avgacc_list=[]
    highestacc_list = []
    lowestacc_list = []
    feats_used = []
    feats_imp = []

    feats = all_features
    while(len(feats) >= 10):
        cv_results = my_cv_scorer(subset,'ridge', feats)
        best_alpha_list.append(cv_results['alpha'])
        avgacc_list.append(cv_results['Average accuracy'])
        highestacc_list.append(cv_results['Highest accuracy'])
        lowestacc_list.append(cv_results['Lowest accuracy'])
        feats_used.append(cv_results['Features used'])
        feats_imp.append(cv_results['Importance'])
        feats = cv_results['Features used'].to_list()[:-1]
    ridge_results_df =  pd.DataFrame({'alpha': best_alpha_list,
                         'Average accuracy': avgacc_list,
                         'Highest accuracy': highestacc_list,
                         'Lowest accuracy': lowestacc_list,
                         'Features used': feats_used,
                         'Importance': feats_imp})
    ridge_results_df.sort_values(by='Lowest accuracy',ascending=False,inplace=True)
    ridge_results_df.sort_values(by='Average accuracy',ascending=False,inplace=True)

    return ridge_results_df

In [64]:
#Same as above for the kNN model, but since the latter does not have a 
#feature importance function (or something analogous to that), to
#determine the order in which we eliminate features, we simply
#use the feature importance obtained from the top scoring ridge model that
#uses all the features.
def knn_selector(subset):
    best_k_list = []
    avgacc_list=[]
    highestacc_list = []
    lowestacc_list = []
    feats_used = []
    
    feats = my_cv_scorer(subset,'ridge')['Features used'].to_list()

    while(len(feats) >= 15):
        cv_results = my_cv_scorer(subset,'knn',feature_list = feats)
        # best_ridges.append(results_df.loc[0,'Model'])
        best_k_list.append(cv_results['k'])
        avgacc_list.append(cv_results['Average accuracy'])
        highestacc_list.append(cv_results['Highest accuracy'])
        lowestacc_list.append(cv_results['Lowest accuracy'])
        feats_used.append(cv_results['Features used'])
        del feats[-1]

    knn_results_df = pd.DataFrame({'k': best_k_list,
                                   'Average accuracy': avgacc_list,
                                   'Highest accuracy': highestacc_list,
                                   'Lowest accuracy': lowestacc_list,
                                   'Features used': feats_used})
    
    knn_results_df.sort_values(by='Lowest accuracy',ascending=False,inplace=True)
    knn_results_df.sort_values(by='Average accuracy',ascending=False,inplace=True)
    
    return knn_results_df

In [65]:
#We do not do any feature selection for the random forest regressor
#This method is included only for naming consistency.
def rfr_selector(subset):
    return my_cv_scorer(subset,'rfr')

In [66]:
#The next method inputs a subset of [0,1,2,3]
#It uses the selector methods to get the "best" models (within the scope
#of the hyperparameters are features we investigate). It fits these models
#to the union of the datasets corresponding to "subset", and it saves
#csv's containing the feature importances and final predictions + results. 
def combined_regressor(subset):

    #Create the main train/test sets
    X_train,y_train = concat_main(subset)
    scaler = StandardScaler()
    scaler.fit_transform(X_train)

    #Used in the filenames of things we save
    name = ''.join(f'_{year_list[i]%100}' for i in subset)

    ####################################################################################
    #Ridge
    start_time = time.monotonic()
    best_ridge = ridge_selector(subset)
    end_time = time.monotonic()
    ridge_time = timedelta(seconds=end_time - start_time)
    alpha = best_ridge.loc[0,'alpha']
    ridge_feats = best_ridge.loc[0,'Features used']
    ridge_cv_acc = best_ridge.loc[0,'Average accuracy']
    print(f'ridge regressor computed in {ridge_time} seconds.')
    print(f'Best CV accuracy {ridge_cv_acc} achieved with alpha = {alpha}.')

    ridge = Ridge(alpha)
    ridge.fit(X_train[ridge_feats],y_train)
    pickle.dump(ridge,open(f'models/ridge{name}.sav', 'wb'))

    fimp_ridge = pd.DataFrame({'feature':ridge_feats,
                            'importance': ridge.coef_,
                            'imp_abs': np.abs(ridge.coef_)}).sort_values(by='imp_abs',ascending=False,ignore_index=True)
    fimp_ridge.drop(inplace=True,columns=['imp_abs'])
    fimp_ridge.to_csv(f'results/fimp_ridge{name}.csv', index=False)

    ####################################################################################
    #knn
    start_time = time.monotonic()
    best_knn = knn_selector(subset)
    end_time = time.monotonic()
    knn_time = timedelta(seconds=end_time - start_time)
    k = best_knn.loc[0,'k']
    knn_feats = best_knn.loc[0,'Features used']
    knn_cv_acc = best_knn.loc[0,'Average accuracy']
    print(f'knn regressor computed in {knn_time} seconds.')
    print(f'best CV accuracy {knn_cv_acc} achieved with k = {k}.')   
    
    knn = KNeighborsRegressor(k)
    knn.fit(X_train[knn_feats],y_train)
    pickle.dump(knn,open(f'models/knn{name}.sav','wb'))

    ####################################################################################
    #random forest
    start_time = time.monotonic()
    best_rfr = rfr_selector(subset)
    end_time = time.monotonic()
    rfr_time = timedelta(seconds=end_time - start_time)
    max_feats = best_rfr['max_features']
    rfr_cv_acc = best_rfr['Average accuracy']
    print(f'random forest regressor computed in {rfr_time} seconds.')
    print(f'best CV accuracy {rfr_cv_acc} achieved with max_features = {max_feats}.')
    
    rfr = RandomForestRegressor(n_estimators=100*len(subset),
                                criterion='squared_error',
                                max_depth=15,
                                max_features=max_feats,
                                n_jobs=-1,
                                bootstrap=True,
                                random_state=42)
    rfr.fit(X_train[all_features],y_train)
    pickle.dump(rfr,open(f'models/rfr{name}.sav','wb'))

    fimp_rfr = pd.DataFrame({'feature': best_rfr['Features used'],
                        'importance': rfr.feature_importances_}).sort_values(by='importance',ascending=False,ignore_index=True)
    fimp_rfr.to_csv(f'results/fimp_rfr{name}.csv', index=False)

    ####################################################################################
    #Assembling the final results
    results_list = []
    comp = list({0,1,2,3} - set(subset))
    for j in comp:
        X_test = X_list[j]
        y_test = y_list[j]
        scaler.transform(X_test)

        y_ridge = ridge.predict(X_test[ridge_feats])
        y_knn = knn.predict(X_test[knn_feats])
        y_rfr = rfr.predict(X_test[all_features])

        #We form a weighted average of the predictions, giving higher
        #weight to the random forest model becuase it tended to perform
        #better in testing.
        y_avg = (y_ridge + y_knn + 2*y_rfr)/4

        pred_df = pd.DataFrame({'ridge':y_ridge,
                                'knn': y_knn,
                                'rfr' : y_rfr,
                                'avg' : y_avg})
        pred_df.to_csv(f'results/{year_list[j]}preds_using{name}.csv',index=False)

        acc_ridge = round(vote_diff_acc(y_ridge,y_test),3)
        acc_knn = round(vote_diff_acc(y_knn,y_test),3)
        acc_rfr = round(vote_diff_acc(y_rfr,y_test),3)
        acc_avg = round(vote_diff_acc(y_avg,y_test),3)

        mse_ridge = round(mse(y_ridge,y_test),3)
        mse_knn = round(mse(y_knn,y_test),3)
        mse_rfr = round(mse(y_rfr,y_test),3)
        mse_avg = round(mse(y_avg,y_test),3)
        
        results_list.append(pd.DataFrame({'Year' : year_list[j],
                                          'Model': [f'Ridge linear (alpha = {alpha})',f'k-Nearest Neighbors (k = {k})',f'Random forest (depth=15,max_features={max_feats})','Averaged'],
                                          'Test accuracy': [acc_ridge,acc_knn,acc_rfr,acc_avg],
                                          'CV accuracy': [ridge_cv_acc,knn_cv_acc,rfr_cv_acc,'N/A'],
                                          'MSE': [mse_ridge,mse_knn,mse_rfr,mse_avg],
                                          'Time taken:': [ridge_time,knn_time,rfr_time, 'N/A']}))
    pd.concat(results_list).to_csv(f'results/results{name}.csv',index=False)

In [67]:
for subset in subset1_list:
    combined_regressor(subset)
    print()

ridge regressor computed in 0:00:48.929804 seconds.
Best CV accuracy 0.853 achieved with alpha = 0.0.
knn regressor computed in 0:02:28.110337 seconds.
best CV accuracy 0.86562 achieved with k = 29.
random forest regressor computed in 0:02:53.301671 seconds.
best CV accuracy 0.94082 achieved with max_features = 0.6.
ridge regressor computed in 0:00:49.847677 seconds.
Best CV accuracy 0.86208 achieved with alpha = 0.0.
knn regressor computed in 0:02:31.345278 seconds.
best CV accuracy 0.87502 achieved with k = 31.
random forest regressor computed in 0:02:55.672485 seconds.
best CV accuracy 0.94257 achieved with max_features = 0.6.
ridge regressor computed in 0:00:49.555368 seconds.
Best CV accuracy 0.79685 achieved with alpha = 0.0.
knn regressor computed in 0:02:36.780210 seconds.
best CV accuracy 0.81044 achieved with k = 39.
random forest regressor computed in 0:02:56.957192 seconds.
best CV accuracy 0.89246 achieved with max_features = 0.6.
ridge regressor computed in 0:00:49.148458

In [68]:
for subset in subset2_list:
    combined_regressor(subset)
    print()

ridge regressor computed in 0:01:06.612525 seconds.
Best CV accuracy 0.77099 achieved with alpha = 0.0.
knn regressor computed in 0:02:39.734586 seconds.
best CV accuracy 0.78461 achieved with k = 29.
random forest regressor computed in 0:09:35.408833 seconds.
best CV accuracy 0.87099 achieved with max_features = 0.4.

ridge regressor computed in 0:01:03.232966 seconds.
Best CV accuracy 0.79929 achieved with alpha = 0.0.
knn regressor computed in 0:02:36.409155 seconds.
best CV accuracy 0.81469 achieved with k = 21.
random forest regressor computed in 0:09:41.745169 seconds.
best CV accuracy 0.89491 achieved with max_features = 0.4.

ridge regressor computed in 0:01:05.099701 seconds.
Best CV accuracy 0.79356 achieved with alpha = 0.0.
knn regressor computed in 0:02:36.695220 seconds.
best CV accuracy 0.8123 achieved with k = 25.
random forest regressor computed in 0:09:43.101404 seconds.
best CV accuracy 0.89408 achieved with max_features = 0.6.

ridge regressor computed in 0:01:04.13

In [ ]:
for subset in [subset3_list]:
    combined_regressor(subset)
    print()